In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import json


warnings.filterwarnings(action='ignore')

## json 원본 데이터 셋 로딩

In [2]:
DATA_DIR = 'C:/workspace/nlp_study/data/Training/신문기사_train_original/'
FILE_NAME = 'train_original.json'

print('json 파일 로딩')
with open(DATA_DIR + FILE_NAME,'r', encoding='UTF8') as json_file:
    json_data = json.load(json_file)

# 읽어온 데이터 셋을 데이터프레임으로 변환

dataset = pd.DataFrame(json_data)

In [3]:
# 데이터프레임의 column 중 documents에 저장되어있는 index와 text값만 활용해야함
# documents 컬럼에는 id, category, media_type, index, text 등 값들이 dict 타입으로 저장되어있음
# 각 key 값을 데이터 프레임의 컬럼으로, value 값은 행으로 만들어주기
print('데이터 프레임 변환')
column_list = dataset['documents'][0].keys()

for column in column_list:
    dataset[column] = dataset['documents'].apply(lambda x : x[column])

In [4]:
dataset.head()

In [5]:
data = dataset.copy()
data.head()

In [6]:
data = dataset[['title','text','extractive','category']]
data

## text 컬럼 안에 있는 리스트에서 index, sentence 추출

In [7]:
sentence_list = []
extractive_target_list = []
category_list = []

for index in tqdm(range(len(data))):

    if 'None' in data['extractive'][index]:
        continue
    else:
        target_list = data['extractive'][index]
    
    for dict_list_index in range(len(data['text'][index])):
        for list_index in range(len(data['text'][index][dict_list_index])):
            text_data = data['text'][index][dict_list_index][list_index]['sentence']
            text_data = re.sub('\[{IMG[0-9][0-9]}]','',text_data)
            text_data = text_data.replace('\'','').replace('\"','')
            
            data_index = data['text'][index][dict_list_index][list_index]['index']

            if data_index in target_list:
                extractive_target_list.append(1)
            else:
                extractive_target_list.append(0)
                
            category_list.append(data['category'][index])
            sentence_list.append(text_data)

In [8]:
print('데이터 저장')
dataset = pd.DataFrame()
dataset['sentence'] = sentence_list
dataset['target'] = extractive_target_list
dataset['category'] = category_list

In [9]:
dataset = dataset.dropna()

In [10]:
dataset.to_csv(path_or_buf = DATA_DIR + 'dataset_final.csv',index=False)